# Segmenting and Clustering Neighborhoods in Toronto

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import time # for time delay while working with API

import requests # library to handle requests

import bs4 # library to parse webpages

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
import geocoder

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# k-means from clustering stage
from sklearn.cluster import KMeans

# Map rendering library
import folium

print('Libraries imported.')

Libraries imported.


## 1. Collecting Neighborhoods

Let's create a webscrapping script to collect Toronto neighborhoods information from the table on https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M with following columns: PostalCode, Borough, and Neighborhood.

In [2]:
# Download the webpage
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
res.raise_for_status()

In [3]:
# Create an beautifulSoup object
toronto_soup = bs4.BeautifulSoup(res.text)

In [4]:
# Selecting all elements inside the corresponding tags
elements = toronto_soup.select('div table tbody tr td')

## 2. Creating Toronto DataFrame

In [6]:
elements[864] 

<td>M9Z</td>

At the previous step we found 289 rows with data. The last postal code in the table is 'M9Z' and its index in __elements__ list is 864.
Let's transform the list, considering assignment rules: 
1. Ignore postal codes with a borough that is 'Not assigned'.
2. If a neighborhood is 'Not assigned', assign it with a corresponding borough.

In [7]:
# Creating a new list of rows
lst = []
for i in range(0, 865, 3):
    postal_code, borough, neighborhood = elements[i].getText(), elements[i+1].getText(), elements[i+2].getText()[:-1]
    if borough == 'Not assigned':
        continue
    if neighborhood == 'Not assigned':
        neighborhood = borough      
    lst.append([postal_code, borough, neighborhood])
lst[:10]

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", "Queen's Park"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern']]

In [8]:
print('Now we have {} rows of relevant data.'.format(len(lst)))

Now we have 212 rows of relevant data.


But there is the third rule: neighborhoods with the same borough and postal code should be combined in the only row. Since the combined rows should not be processed twice, we create __row_to_skip__ list.  
Later we will need a dictionary to convert into a pandas DataFrame.

In [9]:
# Creating a dictionary
toronto_data = {'PostalCode': [], 'Borough': [], 'Neighborhood': []}

# Apply the rule #3 and populate the dictionary
row_to_skip = [False]*len(lst)
for i in range(len(lst)):
    if row_to_skip[i]:
        continue
    p, b, h = lst[i][0], lst[i][1], lst[i][2]
    for j in range(i+1, len(lst)): # start from i+1 because the postal codes in lst is in ascending order
        if row_to_skip[j]:
            continue
        if lst[j][0] == p and lst[j][1] == b:
            h += ', {}'.format(lst[j][2])
            row_to_skip[j] = True
            
    toronto_data['PostalCode'].append(p)
    toronto_data['Borough'].append(b)
    toronto_data['Neighborhood'].append(h)
    
print('toronto_data dictionary has been created!')

toronto_data dictionary has been created!


Now we are ready to achive our first intermediate goal - **Toronto DataFrame**.

In [10]:
toronto_df = pd.DataFrame(data=toronto_data)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
print('Toronto DataFrame has {} data rows.'.format(toronto_df.shape[0]))

Toronto DataFrame has 103 data rows.


## 3. Adding the Geographical Coordinates

Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. We will try 2 approaches: automatic one using a special geo service and "manual" one using the avaliable csv. But firstly we will create a dictionary to store coordinates.

In [12]:
# Creating a dictionary
coordinates = {'Latitude': [], 'Longitude': []}

### 3.1. Geocoder Python Package

Let us run through all postal codes in the Toronto dataframe, retrieve corresponding geographical coordinates, and populate the dictionary.

In [13]:
# Running through Toronto postal codes in the dataframe
for i in range(toronto_df.shape[0]):
    postal_code = toronto_df.iloc[i, 0]
    
    # Call geocoder.geocodefarm as a reliable provider among offered. No while loop because of the strict free request limit - 250 per day.
    g = geocoder.geocodefarm('{}, Toronto, Ontario'.format(postal_code))
    
    # Populate the dictionary
    coordinates['Latitude'].append(g.lat)
    coordinates['Longitude'].append(g.lng)
    time.sleep(0.3)

Status code Unknown from https://www.geocode.farm/v3/json/forward/: ERROR - HTTPSConnectionPool(host='www.geocode.farm', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://www.geocode.farm/v3/json/forward/: ERROR - HTTPSConnectionPool(host='www.geocode.farm', port=443): Read timed out. (read timeout=5.0)


We have to make sure that the geoservice has done what we need. And if everything is normal, we merge created dictionary and Toronto dataframe.

In [14]:
# Check if the dictionary values are all None
if not any(coordinates['Latitude']) or not any(coordinates['Longitude']):
    print('It happens that our automatic approach failed.')
    print(coordinates)
else:
    print('The dictionary has been created successfuly!')

The dictionary has been created successfuly!


### 3.2. Parsing CSV File

If the first aproach fails, and it does that quite often, we use the csv file that icludes all necessary data.

In [15]:
path = '.\Geospatial_Coordinates.csv'
csv_df = pd.read_csv(path)
csv_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# Creating a separate dictionary
coordinates_csv = {'Latitude': [], 'Longitude': []}

Let us run through all postal codes in the Toronto dataframe, retrieve corresponding geographical coordinates from csv_df, and add information into the dictionary.

In [17]:
# Running through toronto dataframe
for i in range(toronto_df.shape[0]):
    postal_code = toronto_df.iloc[i, 0]
    
    # Looking for the postal_code in csv
    for j in range(csv_df.shape[0]):
        if postal_code == csv_df.iloc[j, 0]:
            # Populate the dictionary if found
            coordinates_csv['Latitude'].append(csv_df.iloc[j, 1])
            coordinates_csv['Longitude'].append(csv_df.iloc[j, 2])
print(coordinates_csv)

{'Latitude': [43.7532586, 43.725882299999995, 43.6542599, 43.718517999999996, 43.6623015, 43.6678556, 43.806686299999996, 43.745905799999996, 43.7063972, 43.6571618, 43.709577, 43.6509432, 43.7845351, 43.72589970000001, 43.695343900000005, 43.6514939, 43.6937813, 43.6435152, 43.7635726, 43.67635739999999, 43.644770799999996, 43.6890256, 43.7709921, 43.7090604, 43.6579524, 43.669542, 43.773136, 43.8037622, 43.7543283, 43.7053689, 43.65057120000001, 43.66900510000001, 43.7447342, 43.7785175, 43.7679803, 43.685347, 43.6408157, 43.647926700000006, 43.7279292, 43.7869473, 43.737473200000004, 43.6795571, 43.6471768, 43.6368472, 43.711111700000004, 43.7574902, 43.7390146, 43.6689985, 43.6481985, 43.713756200000006, 43.7563033, 43.716316, 43.789053, 43.7284964, 43.6595255, 43.7332825, 43.6911158, 43.7247659, 43.692657000000004, 43.7701199, 43.7616313, 43.7280205, 43.7116948, 43.67318529999999, 43.706876, 43.7574096, 43.752758299999996, 43.7127511, 43.6969476, 43.6616083, 43.696319, 43.75007150

Make sure that we have the same number of postal codes and corresponding coordinates pairs.

In [18]:
# Comparing the dataframe shape and coordinate lists lengths
if len(coordinates_csv['Latitude']) == len(coordinates_csv['Longitude']) == toronto_df.shape[0]:
    print('Data is consistent.')

Data is consistent.


Convert coordinates dictionary into a new dataframe.

In [19]:
coordinates_df = pd.DataFrame(data=coordinates_csv)
coordinates_df.head()

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494


Lastly we merge it with Toronto dataframe and get our second inermediate result: **FINAL TORONTO DATAFRAME**.

In [20]:
final_toronto_df = pd.concat([toronto_df, coordinates_df], axis=1)
final_toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## 4. Mapping Results

Use geopy library to get the latitude and longitude values of Toronto.

In [21]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="battle_of_neighborhoods")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [22]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_toronto_df['Latitude'], final_toronto_df['Longitude'], final_toronto_df['Borough'], final_toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto)
    
map_toronto

For illustration purposes, let's simplify the above map and segment and cluster only the boroughs that contain the word 'Toronto' in its names. We slice the original dataframe and create a new dataframe of the Toronto data.

In [23]:
# Creating a boolean list
contains_toronto = [False] * final_toronto_df.shape[0]

# Running through all boroughs
for i in range(final_toronto_df.shape[0]):
    if final_toronto_df.iloc[i, 1].endswith('Toronto'):
        # Assign True if a borough ends with 'Toronto'
        contains_toronto[i] = True

print(contains_toronto)    

[False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, True, True, False, False, False, True, True, False, False, False, False, True, True, False, False, False, False, True, True, False, False, False, True, True, True, False, False, False, True, True, False, False, False, False, False, True, False, False, False, False, False, False, True, True, False, False, False, False, True, True, True, False, False, False, True, True, True, False, False, False, True, True, True, False, True, True, False, True, True, False, False, False, True, True, False, False, False, True, True, False, True, True, False, False]


Creating a new lesser dataframe using the boolean list.

In [24]:
toronto_data = final_toronto_df[contains_toronto].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [25]:
print('We got {} different data points.'.format(toronto_data.shape[0]))

We got 38 different data points.


Now let's map them on top of the Toronto map.

In [26]:
# create map of Toronto boroughs using previously obtained latitude and longitude values
map_toronto_only = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto_only)
    
map_toronto_only

## 5. Exploring Neighborhoods in Toronto

Next, we are going to start utilizing the Foursquare API to explore the boroughs and segment them.

Let's create a function that gets the top 100 venues for every neighborhood within a radius of 500 meters.

In [28]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [29]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Check the size of the resulting dataframe.

In [30]:
print('We got {} venues.'.format(toronto_venues.shape[0]))
toronto_venues.head()

We got 1700 venues.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood.

In [31]:
toronto_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()

,Venue
Neighborhood,
"Adelaide, King, Richmond",100
Berczy Park,54
"Brockton, Exhibition Place, Parkdale Village",20
Business Reply Mail Processing Centre 969 Eastern,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14
"Cabbagetown, St. James Town",47
Central Bay Street,81
"Chinatown, Grange Park, Kensington Market",100
Christie,16


Let's find out how many unique categories can be curated from all the returned venues.

In [32]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 unique categories.


## 6. Analyzing Each Neighborhood

Let's analyse each neighborhood venues using one hot encoding.

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[160]] + list(toronto_onehot.columns[:160]) + list(toronto_onehot.columns[161:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,New American Restaurant,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [34]:
toronto_onehot.shape

(1700, 234)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,New American Restaurant,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.030000,0.0000,0.000000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.00,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,

Let's confirm the new size.

In [36]:
toronto_grouped.shape

(38, 234)

Let's print each neighborhood along with the top 5 most common venues.

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4      Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.06
2        Cocktail Bar  0.06
3  Italian Restaurant  0.04
4            Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
                  venue  freq
0        Breakfast Spot  0.10
1                  Café  0.10
2           Coffee Shop  0.10
3  Caribbean Restaurant  0.05
4                   Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2    Recording Studio  0.05
3             Brewery  0.05
4          Smoke Shop  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  fr

Let's put that into a *pandas* dataframe. First, let's write a function to sort the venues in descending order.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Asian Restaurant,Clothing Store,Gym,Bakery,Bar
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Pub,Seafood Restaurant,Steakhouse,Italian Restaurant,Farmers Market,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Grocery Store,Italian Restaurant,Pet Store,Gym,Nightclub,Climbing Gym,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Skate Park,Smoke Shop,Park,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Plane,Boutique,Airport Gate,Airport
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Park,Café,Bakery,Convenience Store,Diner
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Salad Place,Chinese Restaurant,Sandwich Place,Spa,Thai Restaurant
7,"Chinatown, Grange Park, Kensington Market",Bar,Café,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Vietnamese Restaurant,Mexican Restaurant,Chinese Restaurant,Dessert Shop
8,Christie,Grocery Store,Café,Park,Diner,Athletics & Sports,Italian Restaurant,Restaurant,Nightclub,Baby Store,Coffee Shop
9,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Gastropub,Mediterranean Restaurant,Men's Store,Fast Food Restaurant


## 7. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [40]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 3, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [41]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Café,Park,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Hotel,French Restaurant
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Pizza Place,Theater,Plaza,Tea Room,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Hotel,Café,Clothing Store,Italian Restaurant,Bakery,Cosmetics Shop,Park,Breakfast Spot
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Grocery Store,Pub,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Pub,Seafood Restaurant,Steakhouse,Italian Restaurant,Farmers Market,Bakery


Finally, let's visualize the resulting clusters.

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters